In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import cryptocompare
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import missingno as msno
from IPython.display import HTML
from datetime import date
from datetime import timedelta
import datetime
import matplotlib.dates as md
import math
from multiprocessing import Pool

In [2]:
transactions = pd.read_csv('transactions.csv')
transactions['datetime']=pd.to_datetime(transactions['timestamp'],unit='s')
transactions.info()

/usr/local/etc/anaconda3/envs/blockchain/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,13,14,16,23,24,27,28,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745612 entries, 0 to 745611
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   amount                     542103 non-null  float64       
 1   borrowRate                 109770 non-null  float64       
 2   borrowRateMode             109770 non-null  object        
 3   onBehalfOf                 542103 non-null  object        
 4   pool                       745612 non-null  object        
 5   reserve                    738881 non-null  object        
 6   timestamp                  745612 non-null  int64         
 7   user                       745612 non-null  object        
 8   type                       745612 non-null  object        
 9   reservePriceETH            542103 non-null  float64       
 10  reservePriceUSD            542103 non-null  float64       
 11  amountUSD                  542103 non-null  float64 

In [3]:
rates = pd.read_csv('rates.csv')
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563425 entries, 0 to 563424
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   liquidityRate       563425 non-null  float64
 1   reserve             563425 non-null  object 
 2   stableBorrowRate    563425 non-null  float64
 3   timestamp           563425 non-null  int64  
 4   variableBorrowRate  563425 non-null  float64
 5   datetime            563425 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 25.8+ MB


In [4]:
all_users = set(transactions['user'].dropna().unique()).union(transactions['onBehalfOf'].dropna().unique())
len(all_users)

52500

In [5]:
rates['datetime'] = pd.to_datetime(rates['datetime'])

In [19]:
def iterateTransactions(row, assets, liabilities,borrows):
    ttype = row['type']
    token = row['reserve']
        
    if ttype=='deposit':
        if token in assets.keys():
            assets[token]+=row['amount']
        else:
            assets[token]=row['amount']
            
    elif ttype=='redeem':
        if token in assets.keys():
            assets[token]-=row['amount']
            
            #if less than 0, set to 0
            if assets[token]<0:
                assets[token]=0
        else:
            #print('Redeem Error?', token, row['user'], row['protocolContract'], '\n',assets,'\n')
            assets[token]= -row['amount']
            
    elif ttype=='borrow':
        if token in liabilities.keys():
            liabilities[token]+=row['amount']
            
            found = False
            #search for correct borrow entry
            for i in range(len(borrows[token])):
                if row['borrowRateMode']=='Stable' and borrows[token][i][0]=='Stable' and row['borrowRate']==borrows[token][i][1]:
                    borrows[token][i][2]+=row['amount']
                    #print('Adding', row['amount'], token)
                    found = True
                    break
                elif row['borrowRateMode']=='Variable' and borrows[token][i][0]=='Variable':
                    borrows[token][i][2]+=row['amount']
                    #print('Adding', row['amount'], token)
                    #print(borrows[token][i])
                    found = True
                    break
                    
            #if no match, add borrow
            if not found:
                borrows[token].append([row['borrowRateMode'],row['borrowRate'],row['amount']])
                #print('New', row['amount'], token)
            
        else:
            liabilities[token]=row['amount']
            
            #add borrow entry to dictionary for borrows
            borrows[token] = []
            borrows[token].append([row['borrowRateMode'],row['borrowRate'],row['amount']])
            #print(borrows[token][0])
            
    elif ttype=='repay':
        if token in liabilities.keys():
            liabilities[token]-=row['amount']
            #repays don't identify the corresponding borrow, so we remove from the highest interest borrow
            current_amount = row['amount']
            while current_amount>0:
                high_index=0
                for i in range(len(borrows[token])):
                    if borrows[token][i][1]> borrows[token][high_index][1] and borrows[token][i][2]!=0:
                        high_index = i
                
                #check if all borrows have 0 values
                all_zeroes = True
                for borrow in borrows[token]:        
                    if borrow[2]!=0:
                        all_zeroes=False
                
                #if so, exit loop
                if all_zeroes:
                    break
                
                #if outstanding loan greater than repay, update amount and exit loop
                if borrows[token][high_index][2]>current_amount:
                    borrows[token][high_index][2]-=current_amount
                    current_amount=0
                    #print(borrows[token][high_index], current_amount,'first')
                #otherwise, pay off outstanding loan, then find next loan to pay off
                else:
                    current_amount-=borrows[token][high_index][2]
                    borrows[token][high_index][2]=0
                    #print(borrows[token][high_index],current_amount,'second')
            
            #if less than 0, set to 0
            if liabilities[token]<0:
                liabilities[token]=0
                
        else:
            #user shouldn't be able to repay a loan that hasn't been taken out
            print('Repay Error?', token, row['user'], '\n',liabilities,'\n')
            liabilities[token]= -row['amount']
            
    elif ttype=='swap':
        #from variable to stable
        if row['borrowRateModeFrom']=='Variable':
            amount=0
            #get amount for variable loan
            for i in range(len(borrows[token])):
                if borrows[token][i][0]=='Variable' and borrows[token][i][2] !=0:
                    amount = borrows[token][i][2]
                    borrows[token][i][2]=0
                    borrows[token].pop(i)
                    break
                    
            #add new stable loan
   
            borrows[token].append(['Stable',row['stableBorrowRate'],amount])
        #from stable to variable
        else:
            amount=0
            pops=[]
            #get sum of amounts for stable loans
            for i in range(len(borrows[token])):
                if borrows[token][i][0]=='Stable':
                    amount += borrows[token][i][2]
                    borrows[token][i][2]=0
                    pops.append(i)
                    
            #remove loans
            for pop_id in pops:
                borrows[token].pop(pop_id)
                    
            #add new variable loans
            borrows[token].append(['Variable',row['variableBorrowRate'],amount])
            
    elif ttype=='liquidation':
        liabilities[row['principalReserve']]-=row['principalAmount']
        assets[row['collateralReserve']]-=row['collateralAmount']

        
def calculateInterest(date, rates, assets, liabilities, borrows):
    seconds = 0
    #iterate through each asset the user owns
    for asset in assets.keys():
        
        #get rows of correct token for correct day
        rates_token = rates[rates['reserve']==asset]
        rates_time = rates_token[(rates_token['datetime'].dt.year==date.year)&(rates_token['datetime'].dt.month==date.month)&(rates_token['datetime'].dt.day==date.day)]
            
        #get timestamps and liquidity rates
        depositRates=list(rates_time['liquidityRate'])
        depositTimes=list(rates_time['datetime'])
        
        #if no rates for the day, take last availble rate
        if len(depositRates)==0:
            newrates_time = rates_token[(rates_token['datetime'].dt.year<=date.year)&(rates_token['datetime'].dt.month<=date.month)&(rates_token['datetime'].dt.day<=date.day)]
            depositRates = [list(rates_token.sort_values('timestamp')['liquidityRate'])[-1]]
            depositTimes = [datetime.datetime(date.year,date.month,date.day,0,0,0)]

        current_date = datetime.datetime(date.year,date.month,date.day,0,0,0)
        for i in range(len(depositRates)):
            #get elapsed time in days
            elapsed = (depositTimes[i]-current_date).total_seconds()


            elapsed_days = elapsed/86400
            
            #store current date
            current_date = depositTimes[i]
            
            #get rate for time period
            rate = depositRates[i]/100
            daily_rate = rate/365
            current_rate =daily_rate*elapsed_days
            
            assets[asset]+=assets[asset]*current_rate
            
        #calculate interest for rest of day
        end_date = datetime.datetime(date.year,date.month,date.day,23,59,59)
        elapsed = (end_date-current_date).total_seconds()+1
        elapsed_days = elapsed/86400
        
        #get rate for time period
        current_rate =daily_rate*elapsed_days
            
        #update asset amount
        assets[asset]+=assets[asset]*current_rate
    
    #iterate through each asset user owes
    for liability in liabilities.keys():
    
        total_interest=0
        asset_amt = liabilities[liability]
        #iterate through each loan for asset
        for i in range(len(borrows[liability])):
            interest_amount= 0
            
            #if stable loan
            if borrows[liability][i][0]=='Stable':

                #compound for each second throughout day at constant rate 
                interest_amount+=borrows[liability][i][2]*(borrows[liability][i][1]/(100*365))
                
            #if variable loan
            else:
                #get rows of correct token for correct day
                rates_token = rates[rates['reserve']==liability]
                rates_time = rates_token[(rates_token['datetime'].dt.year==date.year)&(rates_token['datetime'].dt.month==date.month)&(rates_token['datetime'].dt.day==date.day)]
                #get timestamps and liquidity rates
                interestRates=list(rates_time['variableBorrowRate'])
                interestTimes=list(rates_time['datetime'])
                
                #if no rates for the day, take last availble rate
                if len(interestRates)==0:
                    newrates_time = rates_token[(rates_token['datetime'].dt.year<=date.year)&(rates_token['datetime'].dt.month<=date.month)&(rates_token['datetime'].dt.day<=date.day)]
                    interestRates = [list(rates_token.sort_values('timestamp')['variableBorrowRate'])[-1]]
                    interestTimes = [datetime.datetime(date.year,date.month,date.day,0,0,0)]
                
                current_date = datetime.datetime(date.year,date.month,date.day,0,0,0)
                for j in range(len(interestRates)):
                    #get elapsed time in days
                    elapsed = (interestTimes[j]-current_date).total_seconds()
                    elapsed_years = elapsed/(86400*365)

                    #store current date
                    current_date = interestTimes[j]

                    #get rate for time period
                    rate = interestRates[j]/100
                    for i in range(len(borrows[liability])):
                        if borrows[liability][i][0] =='Variable':
                            borrows[liability][i][1]=rate*100
                            
                    current_rate =rate*elapsed_years
                    interest_amount +=(borrows[liability][i][2]*current_rate)
                    
                #calculate interest for rest of day
                end_date = datetime.datetime(date.year,date.month,date.day,23,59,59)
                elapsed = (end_date-current_date).total_seconds()+1
                elapsed_years = elapsed/(86400*365)

                #get rate for time period
                current_rate =rate*elapsed_years

                #update asset amount
                interest_amount+=(borrows[liability][i][2]*current_rate)
                     
            borrows[liability][i][2]+=interest_amount
            total_interest+=interest_amount
            
        liabilities[liability]+=total_interest
        #print(seconds)
                
def getUserSummary(transactions, rates, user,lastTimestamp=datetime.datetime.today()):
    print(user)
    #get transactions for user
    monetary = transactions[transactions['onBehalfOf']==user]
    nonmonetary = transactions[(transactions['type'].isin(['liquidation','collateral','swap'])) & (transactions['user']==user)]
    
    df_user = monetary.append(nonmonetary).sort_values('timestamp')

    
    #set up dictionary to keep track of user's tokens             
    assets = dict()
    liabilities = dict()
    borrows=dict()    
    
    start = df_user['datetime'].min()
    end = lastTimestamp
    
    dates = pd.date_range(start,end+timedelta(days=1),freq='d') 
    
    rows=[]
    for date in dates:
        print(date)
        df_day = df_user[(df_user['datetime'].dt.year==date.year) & (df_user['datetime'].dt.month==date.month) & (df_user['datetime'].dt.day==date.day)].sort_values('timestamp')
        
        df_day.apply(lambda x: iterateTransactions(x, assets, liabilities, borrows),axis=1)
        calculateInterest(date,rates,assets,liabilities, borrows)
        
        
        row= dict()
        row['date'] = datetime.date(date.year,date.month,date.day)
        for asset in assets.keys():
            if str(asset)=='nan':
                continue
            row[str(asset)+'_deposited']=assets[asset]
        for liability in liabilities.keys():
            if str(liability)=='nan':
                continue
            row[str(liability)+'_borrowed']=liabilities[liability]
            
        rows.append(row)

    return pd.DataFrame(rows).fillna(0)
'''        
#0xa083d84df0d1d8df2cccf8ee949d0cc0e1ae36e
'''
#getUserSummary(transactions,rates,'0xa083d84df0d1d8df2cccf8ee949d0cc0e1ae36e')
'''
all_summaries = pd.DataFrame()
i=0
for user in all_users:
    user_summary = getUserSummary(transactions,rates,'0x94ee9c600870c4199a1af8496eeb3087f2d1c32f')
    user_summary['user']=user
    all_summaries = all_summaries.append(user_summary)
    i+=1
    if i%1000==0:
        print(str(i)+'/'+str(len(all_users)))
        

all_summaries.info()
'''


def getUserData(user):
    
    user_summary= getUserSummary(transactions,rates,user)
    user_summary['user']=user
        
    return user_summary

'''
results=[]
for user in list(all_users)[1:]:
    results.append(getUserData(user))
'''
'''

i=0
frames=1
df = pd.DataFrame()
user_list = sorted(list(all_users))

for user in user_list:
    try:
        if str(user)=='nan':
            continue
        if len(transactions[transactions['onBehalfOf']==user])==0 or len(transactions[transactions['onBehalfOf']==user])>1000:
            continue
        df = df.append(getUserData(user))
        if i%50==0:
            print(str(i)+'/'+str(len(all_users)))
            df.to_csv('users'+str(frames)+'.csv',index=False)
            df = pd.DataFrame()
            frames+=1
        print(i)
    except:
        print(user)
    i+=1
'''

"\n\ni=0\nframes=1\ndf = pd.DataFrame()\nuser_list = sorted(list(all_users))\n\nfor user in user_list:\n    try:\n        if str(user)=='nan':\n            continue\n        if len(transactions[transactions['onBehalfOf']==user])==0 or len(transactions[transactions['onBehalfOf']==user])>1000:\n            continue\n        df = df.append(getUserData(user))\n        if i%50==0:\n            print(str(i)+'/'+str(len(all_users)))\n            df.to_csv('users'+str(frames)+'.csv',index=False)\n            df = pd.DataFrame()\n            frames+=1\n        print(i)\n    except:\n        print(user)\n    i+=1\n"

In [ ]:
user_list

In [22]:
getUserData('0x4110ea8b4d77ff4c4115cb8f4bbf396804807889').to_csv('ViolaRodriguez.csv',index=False)

0x4110ea8b4d77ff4c4115cb8f4bbf396804807889
2020-12-28 18:21:16
2020-12-29 18:21:16
2020-12-30 18:21:16
2020-12-31 18:21:16
2021-01-01 18:21:16
2021-01-02 18:21:16
2021-01-03 18:21:16
2021-01-04 18:21:16
2021-01-05 18:21:16
2021-01-06 18:21:16
2021-01-07 18:21:16
2021-01-08 18:21:16
2021-01-09 18:21:16
2021-01-10 18:21:16
2021-01-11 18:21:16
2021-01-12 18:21:16
2021-01-13 18:21:16
2021-01-14 18:21:16
2021-01-15 18:21:16
2021-01-16 18:21:16
2021-01-17 18:21:16
2021-01-18 18:21:16
2021-01-19 18:21:16
2021-01-20 18:21:16
2021-01-21 18:21:16
2021-01-22 18:21:16
2021-01-23 18:21:16
2021-01-24 18:21:16
2021-01-25 18:21:16
2021-01-26 18:21:16
2021-01-27 18:21:16
2021-01-28 18:21:16
2021-01-29 18:21:16
2021-01-30 18:21:16
2021-01-31 18:21:16
2021-02-01 18:21:16
2021-02-02 18:21:16
2021-02-03 18:21:16
2021-02-04 18:21:16
2021-02-05 18:21:16
2021-02-06 18:21:16
2021-02-07 18:21:16
2021-02-08 18:21:16
2021-02-09 18:21:16
2021-02-10 18:21:16
2021-02-11 18:21:16
2021-02-12 18:21:16
2021-02-13 18:21:

In [ ]:
import os
os.cpu_count()

In [ ]:
for user in list(all_users)[1:5]:
    print(user)

In [ ]:
rates[rates['reserve']=='ZRX'].describe()

In [21]:
transactions[transactions['onBehalfOf']=='0x4110ea8b4d77ff4c4115cb8f4bbf396804807889'][['datetime','type','reserve','amount','user_alias','onBehalfOf_alias']].sort_values('datetime')

,datetime,type,reserve,amount,user_alias,onBehalfOf_alias
224140,2020-12-28 18:21:16,deposit,LINK,85.843745,Viola Rodriguez,Viola Rodriguez
78984,2020-12-28 18:29:56,borrow,WBTC,0.020000,Viola Rodriguez,Viola Rodriguez
173277,2021-01-02 04:01:44,repay,WBTC,0.020001,Viola Rodriguez,Viola Rodriguez
485126,2021-01-02 04:12:36,redeem,LINK,85.846255,Viola Rodriguez,Viola Rodriguez
249823,2021-01-02 04:21:13,deposit,WETH,7.000000,Dollie Brazil,Viola Rodriguez
286616,2021-01-02 04:24:14,deposit,WBTC,0.227059,Viola Rodriguez,Viola Rodriguez
317972,2021-01-02 04:26:28,deposit,LINK,85.846255,Viola Rodriguez,Viola Rodriguez
13994,2021-01-02 04:30:52,borrow,USDC,9000.000000,Viola Rodriguez,Viola Rodriguez
393657,2021-01-06 06:00:14,deposit,LINK,46.792073,Viola Rodriguez,Viola Rodriguez
237835,2021-01-06 06:03:42,deposit,WETH,3.000000,Dollie Brazil,Viola Rodriguez


In [ ]:
transactions[transa]

In [ ]:
transactions[transactions['user_alias']=='Julian Shearer'][['datetime','type','reserve','amount','borrowRateMode','borrowRate','principalReserve','principalAmount','collateralReserve','collateralAmount']].sort_values('datetime')

In [7]:
transactions['user']

0         0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04
1         0x94ee9c600870c4199a1af8496eeb3087f2d1c32f
2         0x51346d389ec7adf11c5b96695ef003dbb849af9a
3         0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2
4         0x78cbc5e9eb4f96e5ced75819e76634a768b57e07
                             ...                    
745607    0xa083d84df0d1d8df2cccf8ee949d0cc0e1ae36ee
745608    0x4110ea8b4d77ff4c4115cb8f4bbf396804807889
745609    0x498c5431eb517101582988fbb36431ddaac8f4b1
745610    0xdb38ae75c5f44276803345f7f02e95a0aeef5944
745611    0x498c5431eb517101582988fbb36431ddaac8f4b1
Name: user, Length: 745612, dtype: object

In [ ]:
rates[(rates['reserve']=='LINK') & (rates['datetime']>='2021-02-21 04:55:40')].head(100)
    

In [ ]:
depositTimes

In [ ]:
# one period - difference of 0001089
#163.76  apy
#3.54072

#31 536 000

#.00000519/